<a href="https://colab.research.google.com/github/aniketGhetla/FIFA2022_World_Cup_Prediction/blob/main/Predicting_FIFA_2022_Winner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import pandas as pd
from scipy.stats import poisson

In [2]:
dict_table= pickle.load(open('dict_table','rb') )
df_historical_data=pd.read_csv('/content/drive/MyDrive/clean_fifa_worldcup_matches.csv')
df_fixture=pd.read_csv('/content/drive/MyDrive/clean_fifa_worldcup_fixture.csv')

In [3]:
dict_table['Group A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Qatar (H),0,0,0,0,0,0,0,0
1,2,Ecuador,0,0,0,0,0,0,0,0
2,3,Senegal,0,0,0,0,0,0,0,0
3,4,Netherlands,0,0,0,0,0,0,0,0


In [4]:
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Uruguay,Argentina,1930,4,2,6
2,Uruguay,Yugoslavia,1930,6,1,7
3,Argentina,United States,1930,6,1,7
4,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
895,Brazil,Costa Rica,2018,2,0,2
896,Serbia,Switzerland,2018,1,2,3
897,Serbia,Brazil,2018,0,2,2
898,France,Peru,2018,1,0,1


In [5]:
#split data into home and away

df_home= df_historical_data[['HomeTeam','HomeGoals','AwayGoals']]
df_away= df_historical_data[['AwayTeam','HomeGoals','AwayGoals']]

In [6]:
df_away

,AwayTeam,HomeGoals,AwayGoals
0,Mexico,4,1
1,Argentina,4,2
2,Yugoslavia,6,1
3,United States,6,1
4,Belgium,1,0
...,...,...,...
895,Costa Rica,2,0
896,Switzerland,1,2
897,Brazil,0,2
898,Peru,1,0


In [7]:
#rename columns
df_home=df_home.rename(columns ={'HomeTeam':'Team','HomeGoals':'GoalsScored','AwayGoals':'GoalsConceded'})
df_away=df_away.rename(columns ={'AwayTeam':'Team','HomeGoals':'GoalsConceded','AwayGoals':'GoalsScored'})

In [8]:
# concat home and away, groupny team and calculate mean of each team
df_team_strength=pd.concat([df_home,df_away],ignore_index=True).groupby('Team').mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


In [ ]:
#function to predict points returns points each team recieves

def predict_points(home,away):
  if home in df_team_strength.index and away in df_team_strength.index:
    #goalscpred * goals conceeded
    lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
    lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
    prob_home,prob_away,prob_draw = 0,0,0
    for x in range(0,11):
      for y in range(0,11):
        p = poisson.pmf(x,lamb_home) * poisson.pmf(y,lamb_away)
        if x ==y:
          prob_draw += p
        elif x>y:
          prob_home += p
        else:
          prob_away += p
    points_home = 3 * prob_home + prob_draw
    points_away = 3 * prob_away + prob_draw
    return(points_home,points_away)
  else:
    return(0,0)

In [10]:
predict_points('Argentina','Mexico')

(2.3129151525530505, 0.5378377125059863)

In [ ]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [ ]:
#simulate all the matches played in the group stage and update the tables
for group in dict_table:
  teams_in_group = dict_table[group]['Team'].values #names of teams of each group  select the group and the column we want .values gives the team name
  df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]#selects 6 matches each group has to play
  for index, row in df_fixture_group_6.iterrows(): #iterate to each row of the df df_fixture_group_6 that only has 6 matches
    home, away = row['home'], row['away']
    points_home, points_away = predict_points(home, away) #points for each team
    dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home#adding points to the table
    dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away
  dict_table[group] = dict_table[group].sort_values('Pts', ascending = False).reset_index() #sort values in desc order
  dict_table[group] = dict_table[group][['Team', 'Pts']]
  dict_table[group] = dict_table[group].round(0)

In [15]:
dict_table['Group G']

,Team,Pts
0,Brazil,8.0
1,Switzerland,4.0
2,Serbia,3.0
3,Cameroon,2.0


In [16]:
df_fixture_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [29]:
#update the knockout fixtures with group winners and runner ups
for group in dict_table:
  group_winner = dict_table[group].loc[0,'Team']
  runners_up = dict_table[group].loc[1,'Team']
  df_fixture_knockout.replace({f'Winners{group}': group_winner,f'Runners-up{group}':runners_up}, inplace= True)
df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [ ]:
#create winner function
def get_winner(df_fixture_updated):
  for index,row in df_fixture_updated.iterrows():
    home, away = row['home'], row['away']
    points_home, points_away = predict_points(home, away)
    if points_home > points_away:
      winner = home
    else:
      winner = df_away
    df_fixture_updated.loc[index]


In [31]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [32]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


In [33]:
df_fixture_quarter

,home,score,away,year
56,Winners Match 53,Match 58,Winners Match 54,2022
57,Winners Match 49,Match 57,Winners Match 50,2022
58,Winners Match 55,Match 60,Winners Match 56,2022
59,Winners Match 51,Match 59,Winners Match 52,2022


In [ ]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

Quarter Finals

In [ ]:
update_table(df_fixture_knockout, df_fixture_quarter)

In [38]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


Semifinal

In [39]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [40]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


Finals

In [41]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [42]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil
